# PSO algorithm implementation
Enough generic to be used with a lot of optimization problems

You can pass the evaluation, bounding, initialization and logger functions to the fit method 

In [1]:
# Some libraries
from scipy import *
from math import *
from matplotlib.pyplot import *
from functools import *
import sys
from tqdm import tqdm
import time

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Initialization

In [2]:
def initSwarm(nb,dim,eval_func,init_function):
    positions = init_function()
    fits = [eval_func(pos) for pos in positions]
    return [{'vit':[0]*dim, 'pos':positions[i], 'fit':fits[i], 'bestpos':positions[i], 'bestfit':fits[i], 'bestvois':[]} for i in range(nb)]

In [3]:
# Return the particle with the best fitness
def maxParticle(p1,p2,isbetter_func):
    if isbetter_func(p1["fit"], p2["fit"]):
        return p1 
    else:
        return p2

# Returns a copy of the particle with the best fitness in the population
def getBest(swarm,isbetter_func):
    return dict(reduce(lambda acc, e: maxParticle(acc,e,isbetter_func),swarm[1:],swarm[0]))

In [4]:
# Update information for the particles of the population (swarm)
def update(particle,bestParticle,isbetter_func):
    nv = dict(particle)
    if isbetter_func(particle["fit"],particle["bestfit"]):
        nv['bestpos'] = particle["pos"][:]
        nv['bestfit'] = particle["fit"]
    nv['bestvois'] = bestParticle["bestpos"][:]
    return nv

# Calculate the velocity and move a particule
def move(particle, dim, eval_function, bounding_function, psi, cmax):
    nv = dict(particle)

    velocity = [0]*dim
    for i in range(dim):
        velocity[i] = (particle["vit"][i]*psi + \
        cmax*random.uniform()*(particle["bestpos"][i] - particle["pos"][i]) + \
        cmax*random.uniform()*(particle["bestvois"][i] - particle["pos"][i]))

    new_pos = [particle["pos"][i] + velocity[i] for i in range(dim)]
    
    start_time = time.time()
    try:
        position = bounding_function(new_pos)   
    except ValueError:
        position = particle["pos"]

    nv['vit'] = velocity
    nv['pos'] = position
    nv['fit'] = eval_function(position)
    return nv

In [5]:
# MAIN LOOP
def fit(eval_function, 
        bounding_function,
        init_function,
        isbetter_function,
        nb_particle, 
        dim, 
        nb_cycles, 
        psi=0.7,
        cmax=1.47,
        log_function=print,
        log_initialization=False):
    Htemps = []       # temps
    Hbest = []        # distance

    # initialization of the population
    swarm = initSwarm(nb_particle,dim,eval_func=eval_function,init_function=init_function)
    if log_initialization:
        log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="PSO - Initialization", padding=1)
    # initialization of the best solution
    best = getBest(swarm, isbetter_function)
    best_cycle = best

    for i in tqdm(range(nb_cycles)):
        #Update informations
        swarm = [update(e,best_cycle,isbetter_function) for e in swarm]
        # velocity calculations and displacement
        swarm = [move(particle=e, dim=dim, psi=psi, cmax=cmax, eval_function=eval_function, bounding_function=bounding_function) for e in swarm]
        log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="PSO - Swarm", it=i+1)
        # Update of the best solution
        best_cycle = getBest(swarm, isbetter_function)
        if isbetter_function(best_cycle["bestfit"],best["bestfit"]):
            best = best_cycle
            # draw(best['pos'], best['fit'])

        # historization of data
        log_function(best['bestpos'], legend="PSO - Best", text=f"{int(best['bestfit'])}", colors=["green", "blue"], it=i+1, fitness=best['bestfit'])

    #displaying result on the console
    log_function(best['bestpos'], legend="PSO - Best final", text=f"{int(best['bestfit'])}", colors=["green", "blue"], it=nb_cycles, fitness=best['bestfit'])
    return best

## Main test

In [7]:
if __name__ == '__main__':
    def sol_bounding(sol, inf, sup):
        return [min (sup, max (inf, dim_val)) for dim_val in sol]
    
    def logger_function(sol, it, legend, padding, **kwargs):
        if (it % padding == 0):
            if (legend.startswith("Best")):
                printmd(f"### Itération {it} : {legend}")
                print(list(sol))
    
    def initOne(dim,inf,sup,eval_function):
        pos = [random.uniform(inf, sup) for i in range(dim)]
        return pos

    def initialization_function(nb,dim,inf,sup,eval_function):
        return [initOne(dim,inf,sup,eval_function) for i in range(nb)]
    
    def compare_fitness(f1, f2):
        # Minimize or maximize
        return f1 < f2

    dim = 5
    inf, sup = -50, 50
    nb_particle = 20
    nb_cycles = 2000
    psi, cmax = 0.7, 1.47
    
    log_padding = 1000
    
    eval_function = lambda x: reduce(lambda acc,e:acc+e*e,x,0) # sphere
    bounding_function = lambda x: sol_bounding(x, inf=inf, sup=sup)
    log_function = lambda x,it=0,legend=None,padding=log_padding,**kwargs:logger_function(x, it, legend, padding, **kwargs)
    init_function = lambda: initialization_function(nb_particle,dim,inf,sup,eval_function)
    isbetter_function = compare_fitness
                  
    fit(eval_function=eval_function, 
        bounding_function=bounding_function, 
        log_function=log_function,
        init_function=init_function,
        isbetter_function=isbetter_function,
        nb_particle=nb_particle, 
        dim=dim, 
        nb_cycles=nb_cycles,
        psi=psi,
        cmax=cmax)

 43%|████▎     | 856/2000 [00:00<00:00, 2121.75it/s]

### Itération 1000 : Best

 75%|███████▌  | 1501/2000 [00:00<00:00, 2139.55it/s]

[7.757530459705862e-18, -9.173832217713878e-18, -5.015887600967483e-18, 6.236171331127854e-18, -5.841458945762562e-18]


 96%|█████████▋| 1930/2000 [00:00<00:00, 2141.43it/s]

### Itération 2000 : Best

100%|██████████| 2000/2000 [00:00<00:00, 2137.62it/s]

[1.9680723496305343e-37, 2.7257310658443266e-37, 1.3161891808170876e-36, -2.055463825865009e-37, -3.4395822943694714e-37]


### Itération 2000 : Best final

[1.9680723496305343e-37, 2.7257310658443266e-37, 1.3161891808170876e-36, -2.055463825865009e-37, -3.4395822943694714e-37]
